In [379]:
import numpy as np 
import pandas as pd 
import re

In [381]:
patients=pd.read_csv("patients.csv")
treatments=pd.read_csv("treatments.csv",na_values=["-"])
treatment_cut=pd.read_csv("treatments_cut.csv",na_values=["-"])
adverse_reactions=pd.read_csv("adverse_reactions.csv")

In [383]:
patients_df=patients.copy()
treatments_df=treatments.copy()
treatment_cut_df=treatment_cut.copy()
adverse_reactions_df=adverse_reactions.copy()

In [385]:
patients_df["assigned_sex"]=patients_df["assigned_sex"].astype("category")
patients_df["birthdate"]=pd.to_datetime(patients_df["birthdate"])

In [403]:
patients_df=patients_df.drop_duplicates(subset=["given_name","surname"],keep="first")
patients_df["zip_code"].fillna("0.0",inplace=True)
patients_df.loc[8,"given_name"]="David"
patients_df[patients_df["patient_id"]==9]
def zip_code(value):
    return str(value).split(".")[0]
def zip_code_length(value):
    if len(str(value))==5:
        return value 
    else:
        return "0"+str(value)
patients_df["zip_code"]=patients_df["zip_code"]=patients_df["zip_code"].apply(zip_code)
patients_df["zip_code"]=patients_df["zip_code"].apply(zip_code_length)
patients_df["zip_code"]=patients_df["zip_code"].astype("int")
def find_contact_details(text: str) -> tuple:
    # it the value is NaN, then return it
    if pd.isna(text):
        return np.nan
    
    # create the phone number pattern
    phone_number_pattern = re.compile(r"(\+[\d]{1,3}\s)?(\(?[\d]{3}\)?\s?-?[\d]{3}\s?-?[\d]{4})")
    # find the phone number from the value/text, as a result we will get a list
    phone_number  = re.findall(phone_number_pattern, text)

    # if length is 0, then the regex can't find any ph number, then define with NaN
    if len(phone_number) <= 0:
        phone_number = np.nan
    # if the country code is attached with the ph number, for that case, the first
    # element will be the country code and the 2nd element will be the actual ph
    # number. So, get that ph number
    elif len(phone_number) >= 2:
        phone_number = phone_number[1]
    # else, we will get the ph number. Grab it.
    else:
        phone_number = phone_number[0]

    # if we found the ph number (with/without country code), then remove that part from the actual value.
    # after removing the ph number, the remaining string might be the email address.
    possible_email_add = re.sub(phone_number_pattern, "", text).strip()

    # then return the ph number and the email address
    return phone_number, possible_email_add
patients["phone"]=patients_df["contact"].apply(lambda x:find_contact_details(x)).apply(lambda x:x[0])

C:\Users\Shreya Sharma\AppData\Local\Temp\ipykernel_53668\3998321232.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  patients_df["zip_code"].fillna("0.0",inplace=True)


TypeError: 'float' object is not subscriptable

In [359]:
treatments_df=pd.concat((treatments_df,treatment_cut_df))
treatments_df["hba1c_change"]=treatments_df["hba1c_start"]-treatments_df["hba1c_end"]
treatments_df["given_name"]=treatments_df["given_name"].str.title()
treatments_df["surname"]=treatments_df["surname"].str.title()
treatments_df=treatments_df.melt(id_vars=["given_name","surname","hba1c_start","hba1c_end","hba1c_change"],var_name="type",value_name="dosage")
treatments_df["dosage_start"]=treatments_df["dosage"].str.split("-").str.get(0)
treatments_df["dosage_end"]=treatments_df["dosage"].str.split("-").str.get(1)

In [375]:
treatments_df["dosage_start"]=treatments_df["dosage_start"].str.replace("u","")
treatments_df["dosage_end"]=treatments_df["dosage_end"].str.replace("u","")
treatments_df=treatments_df.dropna(how="any")
treatments_df=treatments_df.merge(adverse_reactions,how="left")